<a href="https://colab.research.google.com/github/hamzafarooq/multi-agent-course/blob/main/Module_3/Agentic_RAG/Upload_data_to_Qdrant_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uploading PDF Data to Qdrant with Embeddings

## Introduction

This notebook demonstrates how to process unstructured document data (such as PDF files) and store it in a local vector database using Qdrant. This workflow is useful for building applications like intelligent document search, semantic search engines, or AI-based question-answering systems.

We will start by extracting text content from PDF files, convert that text into numerical representations called embeddings, and finally upload those embeddings into a Qdrant database for efficient retrieval and future use.

In this example, we will use two types of PDF data:
- **OpenAI documentation** (covering tools, APIs, and usage guidelines).
- **10-K financial filings** (official company reports and financial statements by Uber and Lyft).

## Data access

**OpenAI documentation can be found here:** https://platform.openai.com/docs/overview

**10-K financial filings for Uber and Lyft can be found here:** \
1. Uber 2021: https://www.sec.gov/Archives/edgar/data/1543151/000154315122000008/uber-20211231.htm

2. Lyft 2024: https://www.sec.gov/ix?doc=/Archives/edgar/data/0001759509/000175950925000025/lyft-20241231.htm

## Objectives

- **Extract text from PDF files** using the PyMuPDF library.
- **Generate semantic embeddings** for each chunk of text using a model from Nomic from Hugging Face.
- **Store the embeddings in Qdrant**, a vector database running locally.

By the end of this notebook, you'll have a working pipeline that reads documents, encodes them into meaningful vector representations, and persists them in a local database that can be queried later.

For more information about Qdrant, please refer to the following sources:

https://api.qdrant.tech/v-1-13-x/api-reference \
https://api.qdrant.tech/v-1-13-x/api-reference/search/query-points \
https://api.qdrant.tech/v-1-13-x/api-reference/collections/create-collection



## Setup and Dependencies

Before we begin, ensure the necessary libraries are installed and imported


In [1]:
!mkdir /content/10k_files
!mkdir /content/agents

In [4]:
!wget "https://cdn.openai.com/business-guides-and-resources/a-practical-guide-to-building-agents.pdf" -q -O ./agents/agent-guide.pdf

In [2]:
!wget "https://www.dropbox.com/scl/fi/ywc29qvt66s8i97h1taci/lyft-10k-2020.pdf?rlkey=d7bru2jno7398imeirn09fey5&dl=0" -q -O ./10k_files/lyft_10k_2020.pdf
!wget "https://www.dropbox.com/scl/fi/lpmmki7a9a14s1l5ef7ep/lyft-10k-2021.pdf?rlkey=ud5cwlfotrii6r5jjag1o3hvm&dl=0" -q -O ./10k_files/lyft_10k_2021.pdf
!wget "https://www.dropbox.com/scl/fi/iffbbnbw9h7shqnnot5es/lyft-10k-2022.pdf?rlkey=grkdgxcrib60oegtp4jn8hpl8&dl=0" -q -O ./10k_files/lyft_10k_2022.pdf

In [3]:
!wget "https://www.google.com/url?q=https%3A%2F%2Fwww.sec.gov%2FArchives%2Fedgar%2Fdata%2F1543151%2F000154315122000008%2Fuber-20211231.htm" -q -O ./10k_files/uber_10k_2021.pdf

In [5]:
# Install the Qdrant client, which allows you to connect to and interact with a Qdrant vector database.
# Qdrant is often used for similarity search like semantic search or recommendation systems.
!pip install qdrant_client

# Install the Hugging Face Transformers library.
# This library provides pre-trained models for tasks like text embeddings, classification, translation, summarization, and more.
!pip install transformers

# Install the PyMuPDF library (also known as Fitz), which is used for working with PDF files.
# It allows you to extract text, images, and metadata from PDFs,
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 121.1 MB/s eta 0:00:00


In [6]:
# Import necessary libraries

# Import the PyMuPDF library, which is installed as 'fitz'
# This library is used for reading and extracting content from PDF documents.
import fitz

# Import the 'os' module for handling file paths and operating system interactions (not used directly here but often useful)
import os

## 1. Extract Data from PDF Files

In this step, we will use the PyMuPDF library to extract text from our PDF documents. This will allow us to process the content and prepare it for embedding.


In [7]:
def read_text_pymupdf(path):
    """
    Extracts text from a PDF file using the PyMuPDF (fitz) library.

    Parameters:
        path (str): The file path to the PDF document.

    Returns:
        str: A single string containing all the text extracted from the PDF.
    """

    # Open the PDF document using the provided file path
    # This returns a Document object that allows access to each page
    doc = fitz.open(path)

    # Initialize an empty string to collect text from all pages
    text_results = ''

    # Loop through each page in the PDF document
    for page in doc:
        # Extract text content from the current page
        text = page.get_text()

        # Append the extracted text to the cumulative result
        text_results += text

    # Return the complete text from the PDF
    return text_results


In [8]:
# Define the path to the folder containing OpenAI-related PDF documents
document_path_opnai = "/content/agents" #make sure to add your folder path here, this data is available in the github repository for the course

# Define the path to the folder containing 10-K filing PDF documents
document_path_10k = "/content/10k_files" #make sure to add your folder path here, this data is available in the github repository for the course

# Initialize an empty list to hold the extracted text from the OpenAI documents
openai_docs = []

# Initialize an empty list to hold the extracted text from the 10-K documents
docs_10k = []

# Sets to track processed filenames (avoid duplicates)
seen_files_opnai = set()
seen_files_10k = set()

# Loop through each file in the OpenAI documents folder
for _f in os.listdir(document_path_opnai):
    # Process only PDF files and skip duplicate filenames
    if _f.lower().endswith(".pdf") and _f not in seen_files_opnai:
        path = os.path.join(document_path_opnai, _f)
        openai_docs.append(read_text_pymupdf(path))
        seen_files_opnai.add(_f)

# Loop through each file in the 10-K documents folder
for _f in os.listdir(document_path_10k):
    # Process only PDF files and skip duplicate filenames
    if _f.lower().endswith(".pdf") and _f not in seen_files_10k:
        path = os.path.join(document_path_10k, _f)
        docs_10k.append(read_text_pymupdf(path))
        seen_files_10k.add(_f)


In [9]:
docs_10k

['UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n \n(Mark One)\n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2021\nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE TRANSITION PERIOD\nFROM                      TO\nCommission File Number 001-38846\nLyft, Inc.\n(Exact name of Registrant as specified in its Charter)\nDelaware\n20-8809830\n(State or other jurisdiction of\nincorporation or organization)\n(I.R.S. Employer\nIdentification No.)\n185 Berry Street, Suite 5000\nSan Francisco, California\n94107\n(Address of principal executive offices)\n(Zip Code)\nRegistrant’s telephone number, including area code: (844) 250-2773\nSecurities registered pursuant to Section 12(b) of the Act: \nTitle of each class\nTrading\nSymbol(s)\nName of each exchange on which registered\nClass A common stock, par value of $0.00001 per sh

## 2. Prepare Document Chunks with Metadata for Vector Storage

Before we store our documents in a vector database like Qdrant, it's important to organize the data in a meaningful way. This section assigns unique identifiers and metadata to each document chunk.

### Why this step is important:

- **Chunk-Level Tracking**: Each document is split into smaller text chunks to fit the input size of embedding models. Assigning metadata to each chunk helps trace it back to its original source.
  
- **UUID Generation**: By attaching a universally unique identifier (UUID) to each chunk, we ensure every piece of data can be reliably referenced or retrieved later.

- **Metadata Enrichment**: Adding metadata such as the original document path enables better filtering, searching, and organization within the vector database.

This process ensures that once the chunks are embedded and stored, they remain well-organized and easily searchable in downstream applications such as semantic search or document-based Q&A systems.


In [10]:
# Import a text splitter that breaks large texts into smaller, overlapping chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create the text splitter with settings for chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,          # Max characters per chunk
    chunk_overlap=50,         # Overlap between chunks to preserve context
    length_function=len,      # Use Python's len() to measure text length
    is_separator_regex=False, # Treat separators as plain text, not regex
    separators=[              # Preferred breakpoints for splitting
        "\n\n", "\n", " ", ".", ",",
        "\u200b", "\uff0c", "\u3001", "\uff0e", "\u3002", ""
    ],
)

# Split the extracted OpenAI document text into chunks
opnai_chunks = text_splitter.create_documents(openai_docs)

# Split the 10-K documents into chunks as well
chunks_10k = text_splitter.create_documents(docs_10k)

# View the result (list of text chunks)
opnai_chunks


[Document(metadata={}, page_content="A practical \u2028\nguide to \u2028\nbuilding agents\nContents\nWhat is an agent?\n4\nWhen should you build an agent?\n5\nAgent design foundations\n7\nGuardrails\n24\nConclusion\n32\n2\nPractical guide to building agents\nIntroduction\nLarge language models are becoming increasingly capable of handling complex, multi-step tasks. \nAdvances in reasoning, multimodality, and tool use have unlocked a new category of LLM-powered \nsystems known as agents.\nThis guide is designed for product and engineering teams exploring how to build their first agents, \ndistilling insights from numerous customer deployments into practical and actionable best \npractices. It includes frameworks for identifying promising use cases, clear patterns for designing \nagent logic and orchestration, and best practices to ensure your agents run safely, predictably, \u2028\nand effectively.\xa0\nAfter reading this guide, you’ll have the foundational knowledge you need to confide

In [11]:
# Import the uuid module to generate unique identifiers
import uuid

# Loop through each chunk of the OpenAI documents
for i in range(len(opnai_chunks)):
    # Generate a unique ID for each chunk (helps track and reference later)
    unique_id = str(uuid.uuid4())

    # Add metadata to the chunk:
    # 'document_info' stores the source path (where the document came from)
    # 'uuid' stores a unique identifier for the chunk
    opnai_chunks[i].metadata['document_info'] = document_path_opnai
    opnai_chunks[i].metadata['uuid'] = unique_id

# Display the first 5 chunks with their metadata
opnai_chunks[:5]


[Document(metadata={'document_info': '/content/agents', 'uuid': '8303c3ca-7400-4031-9b49-83213b86a665'}, page_content="A practical \u2028\nguide to \u2028\nbuilding agents\nContents\nWhat is an agent?\n4\nWhen should you build an agent?\n5\nAgent design foundations\n7\nGuardrails\n24\nConclusion\n32\n2\nPractical guide to building agents\nIntroduction\nLarge language models are becoming increasingly capable of handling complex, multi-step tasks. \nAdvances in reasoning, multimodality, and tool use have unlocked a new category of LLM-powered \nsystems known as agents.\nThis guide is designed for product and engineering teams exploring how to build their first agents, \ndistilling insights from numerous customer deployments into practical and actionable best \npractices. It includes frameworks for identifying promising use cases, clear patterns for designing \nagent logic and orchestration, and best practices to ensure your agents run safely, predictably, \u2028\nand effectively.\xa0\nAf

In [15]:
len(opnai_chunks)

20

In [12]:
# Loop through each chunk of the 10-K documents
for i in range(len(chunks_10k)):
    # Generate a unique identifier (UUID) for each chunk
    unique_id = str(uuid.uuid4())

    # Assign metadata to each chunk:
    # 'document_info' stores the path of the folder where the 10-K files are located
    # 'uuid' stores the unique identifier for the chunk
    chunks_10k[i].metadata['document_info'] = document_path_10k
    chunks_10k[i].metadata['uuid'] = unique_id

# Display the metadata and content of the 6th chunk (index 5) from the 10-K documents
chunks_10k[5]


Document(metadata={'document_info': '/content/10k_files', 'uuid': '2c3cc872-3c20-4df0-aecc-3b55cba28e25'}, page_content='obligation to update any forward-looking statements made in this Annual Report on Form 10-K to reflect events or circumstances after the date of this Annual Report on\nForm 10-K or to reflect  new information  or the occurrence  of unanticipated  events, except as required by law. We may not actually  achieve the plans, intentions or\nexpectations disclosed in our forward-looking statements and you should not place undue reliance on our forward-looking statements. Our forward-looking statements do\nnot reflect the potential impact of any future acquisitions, mergers, dispositions, joint ventures or investments we may make.\nIn  addition,  statements  that  “we  believe”  and  similar  statements  reflect  our  beliefs  and  opinions  on  the  relevant  subject.  These  statements  are  based  upon\ninformation available to us as of the date of this Annual Report on F

## 3. Embed Chunks for Vector Database

### Purpose:

Embedding the chunks of text is a crucial step in preparing the data for storage in a **vector database** like Qdrant. In this step, we convert each text chunk into a **numerical representation** (vector) that captures its semantic meaning. This allows us to perform advanced operations like **semantic search**, **similarity comparison**, and **retrieval** based on meaning, rather than just keyword matching.

### Why we Embed?

- **Vector Representation**: Embedding transforms text into vectors (lists of numbers) that machine learning models can understand. These vectors represent the **semantic meaning** of the text, allowing similar texts to be grouped together, even if they don’t share exact words.
  
- **Efficient Search**: Storing these vectors in a vector database enables **fast similarity searches**. For example, if you ask a question or search for a document, the vector database can quickly find and return the most relevant results based on the meaning of the text, not just exact matches.
  
- **Contextual Understanding**: The embedding process allows the system to "understand" the context of words, which improves the relevance and accuracy of search results. For example, it helps understand that "machine learning" and "artificial intelligence" are related concepts, even if they don't appear together in the same document.

### What it Does?

- **Transforms text** into dense vectors, capturing the **semantic essence** of the content.
- These vectors are then stored in a **vector database** (Qdrant in our case) for fast retrieval and comparison based on similarity, enabling features like document search or answering questions.
  
In the following steps, we will embed the text chunks using a pre-trained nomic text embed model, and store the resulting vectors in Qdrant for future use.


In [13]:
# Import the necessary libraries from the Hugging Face Transformers library
# AutoTokenizer is used to tokenize the input text into a format that the model can process.
# AutoModel loads the pre-trained model for generating embeddings.
import torch # Import torch for device handling
import numpy as np # For concatenating embeddings
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm # For progress bar

# Load the tokenizer and model from Hugging Face
# 'nomic-ai/nomic-embed-text-v1.5' is a pre-trained model designed for text embeddings
# The `trust_remote_code=True` argument allows the use of the model's code from the remote repository
text_tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
text_model = AutoModel.from_pretrained("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

# Move model to GPU if available for faster inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_model.to(device)
text_model.eval() # Set model to evaluation mode

# Define a function to get embeddings for a given batch of text inputs
def _get_embeddings_batch(texts): # Helper function for a single batch
    """
    Generates embeddings for a single batch of text inputs.

    Parameters:
        texts (list of str): A list of text strings to embed.

    Returns:
        numpy.ndarray: A 2D numpy array where each row is the embedding for a corresponding text.
    """
    if not texts:
        return np.array([])

    with torch.no_grad(): # Disable gradient calculation for inference
        # Tokenize the batch of text inputs
        inputs = text_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
        # Move tokenized inputs to the same device as the model (GPU if available)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Pass the tokenized inputs into the model to get embeddings
        outputs = text_model(**inputs)

        # We use the mean of the last hidden state (output of the final layer) for each token
        # This gives us a fixed-size vector for each text in the batch
        embeddings = outputs.last_hidden_state.mean(dim=1)

        # Return the embeddings as a numpy array, moving them back to CPU if they were on GPU
        return embeddings.cpu().numpy()

def get_document_embeddings_batched(document_chunks, batch_size=32):
    """
    Generates embeddings for a list of document chunks in batches, showing progress.

    Parameters:
        document_chunks (list of Document): A list of Document objects from Langchain.
        batch_size (int): The number of document chunks to process in each batch.

    Returns:
        list of numpy.ndarray: A list where each element is the embedding for a corresponding document chunk.
    """
    all_embeddings = []
    # Extract page_content for batching
    contents = [doc.page_content for doc in document_chunks]

    # Process in batches with a progress bar
    for i in tqdm(range(0, len(contents), batch_size), desc="Generating Embeddings"):
        batch_contents = contents[i:i + batch_size]
        batch_embeds = _get_embeddings_batch(batch_contents)
        all_embeddings.extend(batch_embeds)
    return all_embeddings

# Example usage (updated for batch processing and progress bar)
# This example now simulates processing a list of Document objects.
# In actual usage, this `get_document_embeddings_batched` function will be called
# directly with `opnai_chunks` and `chunks_10k` in cell `5ms4hviWkf37`.
class MockDocument: # A mock class to simulate langchain Document objects
    def __init__(self, page_content):
        self.page_content = page_content

sample_docs = [
    MockDocument("This is the first document chunk."),
    MockDocument("This is the second document chunk, with more detail."),
    MockDocument("The third chunk is here."),
    MockDocument("And finally, the fourth document chunk comes into play."),
    MockDocument("Another chunk to demonstrate batching."),
    MockDocument("The final piece of text."),
]
print("--- Example of batched embedding generation ---")
embeddings_example = get_document_embeddings_batched(sample_docs, batch_size=2)

# Print the shape of the embeddings (should be num_docs x embedding_dim)
print(f"Shape of example embeddings: {np.array(embeddings_example).shape}")
if embeddings_example:
    print(f"First 5 values of the first example embedding: {embeddings_example[0][:5]}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

--- Example of batched embedding generation ---


Generating Embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Shape of example embeddings: (6, 768)
First 5 values of the first example embedding: [ 0.69545746  0.5865303  -3.6723402  -1.8932141   0.43078858]


In [14]:
# Embed the OpenAI document chunks into vectors using the `get_document_embeddings_batched` function
# This function processes chunks in batches and shows a progress bar.
opnai_texts_embeded = get_document_embeddings_batched(opnai_chunks)

# Embed the 10-K document chunks into vectors using the `get_document_embeddings_batched` function
# This also processes chunks in batches and shows a progress bar.
texts_embeded_10k = get_document_embeddings_batched(chunks_10k)

Generating Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Embeddings:   0%|          | 0/36 [00:00<?, ?it/s]

##  4. Initialize Qdrant

### Purpose:

This step initializes **Qdrant**, a vector database, to store the text embeddings generated earlier. Qdrant is designed for high-performance similarity search and retrieval of vector data, making it ideal for tasks like semantic search, recommendation systems, or nearest neighbor searches.

### Why this is important?

- **Qdrant Initialization**: This code sets up the Qdrant database to store vectors efficiently.
- **Creating a Collection**: Collections are like tables in a relational database, and in this case, the collection stores vectors representing text embeddings.
- **Cosine Similarity**: Using cosine similarity enables the database to quickly identify vectors (texts) that are semantically similar to a given query.
  
Once the collection is created and initialized, it will be ready to store the embeddings from the OpenAI documents and 10-K filings, making it possible to perform fast similarity searches later.



In [20]:
# Import necessary libraries from the Qdrant client
# QdrantClient is used to interact with the Qdrant database.
# models provides predefined parameters like vector configurations.
from qdrant_client import models
from qdrant_client import QdrantClient, models

import qdrant_client
import os

qdrant_data_dir = '/content/qdrant_data'

# Remove the directory if it exists
if os.path.exists(qdrant_data_dir):
    shutil.rmtree(qdrant_data_dir)
    print(f"Removed existing Qdrant data directory: {qdrant_data_dir}")
else:
    print(f"Qdrant data directory does not exist: {qdrant_data_dir}")


# Create the directory if it doesn't already exist
# `exist_ok=True` ensures no error is raised if the directory already exists
os.makedirs(qdrant_data_dir, exist_ok=True)

# Initialize the Qdrant client, passing the path where the database will be stored
# The Qdrant client will manage operations like adding vectors, creating collections, and querying data
client = qdrant_client.QdrantClient(path=qdrant_data_dir)


Qdrant data directory does not exist: /content/qdrant_data


In [21]:
# Determine the size (dimensionality) of the embeddings by checking the length of the first embedding vector
# The embedding size is the number of values in each vector representation of the text (e.g., 768, 1024, etc.)
text_embeddings_size = len(opnai_texts_embeded[0])

# Check if the collection "opnai_data" already exists in Qdrant
# If it does not exist, the code proceeds to create a new collection
if not client.collection_exists("opnai_data"):
    # Create a new collection in Qdrant to store the OpenAI document embeddings
    client.create_collection(
        # Name of the collection, which helps identify it in the database
        collection_name="opnai_data",

        # Define the vector configuration (size and distance metric) for the collection
        vectors_config=models.VectorParams(
            size=text_embeddings_size,  # Set the vector size to match the embeddings' dimensionality
            distance=models.Distance.COSINE,  # Use Cosine similarity to measure distance between vectors
        ),
    )


In [22]:
# Check if the collection "10k_data" already exists in Qdrant
# If it does not exist, the code proceeds to create a new collection
if not client.collection_exists("10k_data"):
    # Create a new collection in Qdrant to store the 10-K document embeddings
    client.create_collection(
        # Name of the collection, which will hold the 10-K document embeddings
        collection_name="10k_data",

        # Define the vector configuration (size and distance metric) for the collection
        vectors_config=models.VectorParams(
            size=text_embeddings_size,  # Set the vector size to match the embeddings' dimensionality
            distance=models.Distance.COSINE,  # Use Cosine similarity to measure distance between vectors
        ),
    )


## 5. Store Embeddings in Qdrant

### Purpose:

In this step, we insert the embeddings (numerical representations of text) for both the OpenAI documents and the 10-K filings into their respective collections in Qdrant. By storing these embeddings in Qdrant, we can enable efficient semantic search functionality, which is a key part of the **Agentic RAG** (Retriever-Augmented Generation) project.

Once the embeddings are stored, we can leverage Qdrant's similarity search capabilities to retrieve semantically relevant documents based on a given query.


In [23]:
# Define the names of the Qdrant collections we're working with
# These are the collections where document embeddings will be stored
clusters = ["opnai_data", "10k_data"]

# Import numpy to handle vector data as arrays
import numpy as np

# Upload (store) the embeddings of OpenAI documents into the 'opnai_data' collection in Qdrant
client.upload_points(
    collection_name="opnai_data",  # Name of the target collection in Qdrant

    # Create a list of PointStruct objects, one for each embedded document chunk
    points=[
        models.PointStruct(
            id=doc.metadata['uuid'],  # Use the pre-generated UUID as a unique ID for each point (chunk)

            vector=np.array(opnai_texts_embeded[idx]),  # The embedding vector for the chunk, converted to a NumPy array

            payload={  # Payload stores extra information (metadata and content) along with the vector
                "metadata": doc.metadata,       # Include metadata like document path and UUID
                "content": doc.page_content     # Store the original text content of the chunk
            }
        )
        for idx, doc in enumerate(opnai_chunks)  # Loop through all OpenAI chunks and embed them
    ]
)


In [24]:
# Upload (store) the embeddings of 10-K documents into the '10k_data' collection in Qdrant
client.upload_points(
    collection_name="10k_data",  # Target collection name in Qdrant

    # Create a list of points, one for each chunk in the 10-K documents
    points=[
        models.PointStruct(
            id=doc.metadata['uuid'],  # Unique ID for each vector (from previously assigned UUID)

            vector=np.array(texts_embeded_10k[idx]),  # The embedding vector for the chunk, converted to NumPy array

            payload={  # Payload contains additional information for each vector
                "metadata": doc.metadata,       # Include metadata such as source path and UUID
                "content": doc.page_content     # Include the original chunk text for retrieval/display
            }
        )
        for idx, doc in enumerate(chunks_10k)  # Loop through all 10-K chunks to create PointStructs
    ]
)


In [27]:
def search_qdrant(query_text, collection_name, top_k=5):
    """
    Performs a semantic search on a specified Qdrant collection.

    Args:
        query_text (str): The text of the query.
        collection_name (str): The name of the Qdrant collection to search.
        top_k (int): The number of top results to return.

    Returns:
        list: A list of search results from Qdrant.
    """
    # Generate embedding for the query text
    query_embedding = _get_embeddings_batch([query_text])[0]

    # Perform search using the Qdrant client
    # The 'query' parameter now directly accepts the vector.
    search_results = client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=top_k,
        with_payload=True,
    )
    # Access the 'points' attribute as suggested by the user's sample code
    return search_results.points

# --- Example Usage ---
search_query = "What is an AI agent?"

print(f"\nSearching OpenAI documentation for: '{search_query}'")
openai_results = search_qdrant(search_query, "opnai_data", top_k=3)
for i, hit in enumerate(openai_results):
    print(f"Result {i+1} (score: {hit.score:.2f}):")
    print(f"  Content: {hit.payload['content'][:200]}...") # Print first 200 chars
    print(f"  Source: {hit.payload['metadata']['document_info']}")
    print("---")

search_query_10k = "What are the main risks for Lyft?"

print(f"\nSearching 10-K filings for: '{search_query_10k}'")
k10_results = search_qdrant(search_query_10k, "10k_data", top_k=3)
for i, hit in enumerate(k10_results):
    print(f"Result {i+1} (score: {hit.score:.2f}):")
    print(f"  Content: {hit.payload['content'][:200]}...") # Print first 200 chars
    print(f"  Source: {hit.payload['metadata']['document_info']}")
    print("---")


Searching OpenAI documentation for: 'What is an AI agent?'
Result 1 (score: 0.66):
  Content: Agent design 
foundations
In its most fundamental form, an agent consists of three core components:
01
Model
The LLM powering the agent’s reasoning and decision-making
02
Tools
External functions or A...
  Source: /content/agents
---
Result 2 (score: 0.65):
  Content: A practical  
guide to  
building agents
Contents
What is an agent?
4
When should you build an agent?
5
Agent design foundations
7
Guardrails
24
Conclusion
32
2
Practical guide to building agents
Intr...
  Source: /content/agents
---
Result 3 (score: 0.65):
  Content: from
import
 agents 
 Agent, Runner          

technical_support_agent = Agent(
    name=
    instructions=(
        
    ),
    tools=[search_knowledge_base]
)

sales_assistant_agent = Agent(
    nam...
  Source: /content/agents
---

Searching 10-K filings for: 'What are the main risks for Lyft?'
Result 1 (score: 0.76):
  Content: press  releases,  public  confere

## Final Notes

At this point, we've successfully completed the full pipeline for preparing and storing document embeddings:

1. Extracted text from PDFs using PyMuPDF.
2. Split the documents into manageable text chunks.
3. Embedded each chunk using the `nomic-embed-text` model.
4. Stored the resulting vectors, along with metadata and original content, into Qdrant collections (`opnai_data` and `10k_data`).

These embeddings are now stored in a local Qdrant vector database located at:

/content/qdrant_data


You can reuse this vector database in any other project by pointing to the same path. In our case, we will be using it as the retrieval layer in the **Agentic RAG** system, where relevant chunks will be fetched based on user queries and passed to an LLM to generate meaningful, grounded responses.

This setup forms the foundation for building retrieval-augmented applications.
